In [39]:
#!/usr/bin/env python
import sys, re

# Eric Chen 11381898 11/30/2015

# This submission is based on the template interpreter:
# A simple postscript interpreter. It provides only minimal error
# checking.
# CptS 355 students have permission to use this code as the basis
# for their SSPS interpreter (not for the SPS interpreter!)
# PROVIDED that these header comments are left in the file,
# and that you replace the "Put your name" line above
# with your own name and the date (month and year is sufficient)
#
# Carl Hauser
# 27 October 2006
# Slightly improved and updated for Python3 9 Nov 2014

# Lightly referenced: https://msujaws.wordpress.com/2011/05/03/static-vs-dynamic-scoping/
#                      https://github.com/jessechisel126/ssps/blob/master/ssps.py

# Design decisions:
#    the stack is implemented as a python list using s.pop and s.append for pop and push
#    the dictStack is implemented as python list with the "warm" end at 0
#    postscript names represented as python strings and do not include the leading /
#    code arrays are represented as lists containing tokens, not including the surround braces
#    postscript true and false are represented as python True and False
#    postscript numbers are represented as python float values

# Define a separate function for each arithmetic and boolean 
# primitive function

opstack = []
dictStack = [{}] # dictStack. Used for dynamic scoping
ss_stack = []

index = 0

# Input String:
s = "-s /x 4 def /g { x stack } def /f { /x 7 def g } def f"

# Arithmetic ops: each take two numbers and return a number
def plus(x,y):
    return x+y

def minus(x,y):
    return x-y

def times(x,y):
    return x*y

def divide(x,y):
    return x/y

# Comparisons take two numbers and return a boolean
def eq(x,y):
    return x==y

def lt(x,y):
    return x<y

def gt(x,y):
    return x>y

def le(x,y):
    return x<=y

def ge(x,y):
    return x>=y

# Boolean operators: define all the boolean operators in this cell -- and, or, not
# Logical bitwise and (both must be 1 to be true) (From sps interpreter)
def andd(x,y):
    return(x and y)
# Logical bitwise or (one must be 1 to be true)
def orr(x,y):
    if (x or y):
        opstack.append(True)
    else:
        opstack.append(False)

# Reverses whatever the value on opstack was
def nott():
    op1 = opstack.pop()
    print(" ")
    print("=============")
    if(not op1):
        opstack.append(False)
    else:
        opstack.append(True)
        
# HELPER FUNCTION 1
def detect_Mode():
    if '-s' in s:
        return False
    else:
        return True

# HELPER FUNCTION 2, searches the ss_stack for a specified dictionary
def ss_Search(x):
    i=0
    length = len(ss_stack)

    while(i < length):
        (index, tmp) = ss_stack[i]
        if (x in tmp.keys()): return True
        else:
            i += 1
    return False

# HELPER FUNCTION 3, searches the ss_stack for a specified dict value
def ss_Index(x):
    i=0
    length = len(ss_stack)

    while(i < length):
        (index, tmp)=ss_stack[i]
        if (x in tmp.keys()): return index
        else:
            i +=1
    return None

# Map the postscript operator names to the above functions
# using a dictionary
binops={
    "add": plus,
    "sub": minus,
    "mul": times,
    "div": divide,
    "eq": eq,
    "lt": lt,
    "gt": gt,
    "le": le,
    "ge": ge,
    "and": andd,
    "or": orr,
    "not": nott,
}

# This is the global operand stack
stack = []

# and this the global dictionary stack
dictStack = [{}]
    
def printStack():
    # Note: this DOES meet the requirements of the assignment :D
    is_static = detect_Mode()
    
    print ("==============")
    for item in reversed(opstack):
        print (item)
    print ("==============")
    
    if (is_static == False): 
        for x in reversed(ss_stack):
            print (x[1])
    else:
        for x in reversed(dictStack):
            for y in x: 
                print (y, x[y])
    print ("==============")

# We need to recognize postscript names, which start with
# slash and a letter, followed by other characters
name = re.compile('/[a-zA-Z?].*')

# look for name t in the dictionary stack, returning a boolean
# Note the assumption that the warmest end of dictStack is at 
# position 0.
def defined(t):
    is_static = detect_Mode()
    
    if(is_static == True): 
        # note that dictStack is non-local in this function
        for d in dictStack:
            if t in d.keys(): return True
    elif(is_static == False): 
        if(len(ss_stack)>0): 
            (index, tmp) = ss_stack[0]
            finder = ss_Search(t);
            return finder
    else: return False

# look for name t in the dictionary stack, returning either
# its defined value or None if not found
def definition(t):
    is_static = detect_Mode()
    
    if(is_static == True):
        for d in dictStack:
            if t in d.keys():
                return d[t]
    elif(is_static == False):
        if(len(ss_stack)>0):
            index = ss_Index(t)
            if(index != None):
                (linkIndex, linkDict) = ss_stack[index]
                return linkDict[t]
            else:
                return None
    else:
        return None

# Having seen a { read input to the
# matching }, returning the code between the braces
# and the position following the }
def readCode(tokens, p):
    code = []
    braceCount = 1
    # it is not clear how best to use a for loop here.
    # invariant: p points at the next token to be read
    while True:
        t = tokens[p]
        p += 1
        if t=='}':
            braceCount = braceCount - 1
            if braceCount==0: 
                break
        elif t=='{':
            braceCount = braceCount + 1
        code.append(t)
    # invariant tells us this is the correct value of  p to return
    return code, p
    
# The main guts of the interpreter
def evalLoop(tokens):
    global opstack, dictStack
    global index
    
    p = 0
    # invariant: p points one position past the last
    # token processed, whether that is the token after t
    # or after a code array beginning at t
    
    is_static = detect_Mode()
    while p < len(tokens):
        t = tokens[p]
        p += 1
        
        # handle the binary operators
        if t in binops.keys():
            op = binops[t]
            if len(stack)>1:
                dictStack.append({})
                print("dictStack: ", dictStack)
                opstack[-3:] = [op(opstack[-2],opstack[-1])]
                dictStack.pop()
                print("dictStack:: ", dictStack)
            else:
                print ("not enough operands for"), t

        # handle an opening brace - read to the 
        # matching brace and push the resulting code array
        # on the operand stack
        elif t=='{':
            code, p = readCode(tokens, p)
            opstack.append(code)

        # the stack operations exch, pop, clear are easy
        # should add error checking
        elif t=='exch':
            opstack[-2:] = [opstack[-1],opstack[-2]]
        elif t=='pop':
            opstack.pop()
        elif t=='clear':
            opstack = []
        elif t=='stack':
            printStack()
        elif t=='true':
            opstack.append(True)
        elif t=='false':
            opstack.append(False)

        # handle def
        elif t=='def':
            if len(opstack)>1 and type(opstack[-2]) == type('') and (is_static == True):
                # dynamic definition rules
                dictStack[0][opstack[-2]] = opstack[-1]
                opstack[-2:] = []
            elif len(opstack)>1 and type(opstack[-2]) == type('') and (is_static == False):
                # static definition rules
                tmp = {opstack[-2]:opstack[-1]}
                opstack[-2:] = []
                ss_stack.append((index, tmp))  
                index +=1
            else: print ("invalid operands for", t)

        # handle if
        elif t=='if':
            if len(opstack)>1 and type(opstack[-1])==type(True) and type(opstack[-2])==type([]):
                code = opstack.pop()
                cond = opstack.pop()
                if cond:
                    # recursively process the true branch code
                    evalLoop(code)
            else: print ("invalid operands for" , t)

        # handle ifelse  
        elif t=='ifelse':
        # ifelse is similar but takes two code arrays as args
            if (len(opstack)>1 and type(opstack[-3])==type(True) and type(opstack[-2])==type([]) and type(opstack[-1])==type([])):
                elsecode = opstack.pop()
                ifcode = opstack.pop()
                cond = opstack.pop()
                if cond:
                    code = ifcode
                else:
                    code = elsecode
                evalLoop(code)
            else: print ("invalid operands for" , t)

        # Use regular expression match to see if the token
        # t is a name constant, e.g. /abc123; if so push it
        elif name.match(t):
            opstack.append(t[1:])

        # Is it a name defined in a postscript dictionary?
        elif defined(t):
            defn = definition(t)
            # if the definition is a code array it needs to be
            # called so recursively call evalLoop
            if(type(defn)==type([])):
                evalLoop(defn)
            else:
            # otherwise, just push the value on the op stack
                opstack.append(defn)
        else:
        # if nothing else works try to interpret it as a number
            try:
                # if it is a number push it
                opstack.append(float(t))
            except:
            # otherwise, it's an error
            #     print (t,  " is not a valid token") commented out because all it catches is:
                  error = True
            #     -  is not a valid token
            #     s  is not a valid token
    return

pattern = '/?[a-zA-Z][a-zA-Z0-9_]*|[-]?[0-9]+|[}{]+|%.*|[^\t\n ]'

def parse(s):
    tokens = re.findall(pattern, s)
    evalLoop(tokens)

if __name__ == "__main__":

    parse(str(s))
    #printStack()

4.0
{'x': 7.0}
{'f': ['/x', '7', 'def', 'g']}
{'g': ['x', 'stack']}
{'x': 4.0}
